In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

url ='https://docs.cpuc.ca.gov/advancedsearchform.aspx'
proceeding_id = 'r2211013'

driver = webdriver.Chrome()
driver.get(url) 

proceeding_num = driver.find_element(By.ID, "ProcNum")
proceeding_num.send_keys(proceeding_id)      # Filling in the search term in the corresponding input text field
search_button = driver.find_element(By.ID, "SearchButton")
search_button.click()     # Initiating the search


In [ ]:
i = 1

while True:  
    
    all_results = driver.find_elements("link text", "PDF") 
    # Retrieve all Selenium webElements that contains links to the pdf docoments to be downloaded

    pdf_url_list = []
    for result in all_results:
        pdf_url_list.append(result.get_attribute('href')) # Generate a list of urls to download the pdf documents from

    # Requests URL and get response object
    for pdf_url in pdf_url_list:
        pdf_response = requests.get(pdf_url) # retrieve the pdf document content from the server

        url_list_tmp = pdf_url.split('/')
        pdf_id = url_list_tmp[-1]  # extract the name of the pdf file from the url

        with open(pdf_id, 'wb') as pdf_handle:
            pdf_handle.write(pdf_response.content) # write the pdf content into a file 
        
    print(f'Page {i} pdf files have finished downloading!')
    i += 1   # Update counter to track which page we are currently downloading from 
    
    if driver.find_elements(By.ID, 'lnkNextPage'): # check if the "Next Page" link exists (not the last page)
    # webdriver find_elements methods do not raise an error when an item is not found. 
    # Instead it generates an empty list which would evaluate as False
        driver.find_element(By.ID, 'lnkNextPage').click() # Navigate to the next page 
    else:
        print("This is the last page for all the found pdf files!")
        break